**Importing all the necessary packages**

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
from sklearn.preprocessing import LabelEncoder
LabE = LabelEncoder()
from sklearn.preprocessing import StandardScaler
std_scal = StandardScaler()
from sklearn.model_selection import train_test_split as ts
from keras.models import Sequential 
seq = Sequential()
from keras.layers import Dense,Activation,Dropout

**Collecting and reading the data set**

In [ ]:
train_data = pd.read_csv('../input/san-francisco-crime-classification/train.csv')
train_data.head()

In [ ]:
test_data = pd.read_csv('../input/san-francisco-crime-classification/test.csv')
test_data.head()

In [ ]:
sam_sub = pd.read_csv('../input/samplesubmission1/sampleSubmission.csv')
sam_sub.head()

In [ ]:
sam_sub.columns.value_counts().sum()

**creating a new data column with the title year in train_data dataframe**

In [ ]:
type(train_data['Dates'])

In [ ]:
train_data['year'] = train_data['Dates'].apply(lambda x : x.split()[0].split('-')[0])
train_data.head()

**creating a new data column with the title Month in train_data dataframe**

In [ ]:
train_data['Month'] = train_data['Dates'].apply(lambda x : x.split()[0].split('-')[1])
train_data.head()

**creating a new data column with the title Day in train_data dataframe with the help of dates column and creating a new dataset named timing_train.**

In [ ]:
train_data['Day'] = train_data['Dates'].apply(lambda x : x.split()[0].split('-')[2])
timing_train = train_data['Dates'].apply(lambda x : x.split()[1].split(':')[:2])
timing_train

In [ ]:
train_data['timing'] = [int(x[0])*60 + int(x[1]) for x in timing_train]
train_data.head()

In [ ]:
train_data.drop('Dates',axis = 1, inplace = True)

**Adding all the columns of train_data in test_data.**

In [ ]:
test_data['Years'] = test_data['Dates'].apply(lambda x : x.split()[0].split('-')[0])
test_data['Months'] = test_data['Dates'].apply(lambda x : x.split()[0].split('-')[1])
test_data['Days'] = test_data['Dates'].apply(lambda x : x.split()[0].split('-')[2])
timing_test = test_data['Dates'].apply(lambda x : x.split()[1].split(':')[:2])
test_data['timing'] = [int(x[0])*60 + int(x[1]) for x in timing_test]
test_data.head()

In [ ]:
test_data.drop('Dates',axis = 1, inplace = True)

**Plotting bar graphs of necessary data.**

In [ ]:
train_data.PdDistrict.value_counts().plot(kind = 'bar',figsize = (13,13))
plt.show()

In [ ]:
train_data.DayOfWeek.value_counts().plot(kind = 'bar',figsize = (13,13))
plt.show()

In [ ]:
train_data['Category'].value_counts().plot(kind = 'bar',figsize = (13,13))
plt.ylabel('numbers')
plt.xlabel('Crime_category')
plt.show()

**Replacing days of week with the numbers.**


In [ ]:
train_data['DayOfWeek'].replace(to_replace=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'],value=[i for i in range(0,7)],inplace=True)
test_data['DayOfWeek'].replace(to_replace=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'],value=[i for i in range(0,7)],inplace=True)
train_data.head()

In [ ]:

train_data['DayOfWeek'] = LabE.fit_transform(train_data['DayOfWeek'])
train_data.head()

In [ ]:
test_data['DayOfWeek'] = LabE.fit_transform(test_data['DayOfWeek'])
test_data.head()

In [ ]:
train_data = pd.concat([train_data,pd.get_dummies(train_data['PdDistrict'])],axis = 1)
train_data.drop('PdDistrict',axis = 1, inplace = True)
train_data.head()

In [ ]:
test_data = pd.concat([test_data,pd.get_dummies(test_data['PdDistrict'])],axis = 1)
test_data.drop('PdDistrict',axis = 1, inplace = True)
test_data.head()

In [ ]:
train_data.drop(['Descript','Resolution','Address'],axis = 1, inplace = True)
test_data.drop('Address',axis = 1, inplace = True)


In [ ]:
train_data[train_data["Y"] == max(train_data["Y"])]

**Implementing PCA (Principal Component Analysis)**

In [ ]:


std_scal.fit(train_data[["X","Y","timing"]]) 
train_data[["X","Y","timing"]]  = std_scal.transform(train_data[["X","Y","timing"]])
train_data.head()

In [ ]:
std_scal.fit(test_data[["X","Y","timing"]]) 
test_data[["X","Y","timing"]]  = std_scal.transform(test_data[["X","Y","timing"]])
test_data.head()

In [ ]:
train_data["rot45_X"] = .707* train_data["Y"] + .707* train_data["X"]
train_data["rot45_Y"] = .707* train_data["Y"] - .707* train_data["X"]

train_data["rot30_X"] = (1.732/2)* train_data["X"] + (1./2)* train_data["Y"]
train_data["rot30_Y"] = (1.732/2)* train_data["Y"] - (1./2)* train_data["X"]

train_data["rot60_X"] = (1./2)* train_data["X"] + (1.732/2)* train_data["Y"]
train_data["rot60_Y"] = (1./2)* train_data["Y"] - (1.732/2)* train_data["X"]

train_data["radial_r"] = np.sqrt( np.power(train_data["Y"],2) + np.power(train_data["X"],2) )
train_data.head()

In [ ]:
test_data["rot45_X"] = .707* test_data["Y"] + .707* test_data["X"]
test_data["rot45_Y"] = .707* test_data["Y"] - .707* test_data["X"]

test_data["rot30_X"] = (1.732/2)* test_data["X"] + (1./2)* test_data["Y"]
test_data["rot30_Y"] = (1.732/2)* test_data["Y"] - (1./2)* test_data["X"]

test_data["rot60_X"] = (1./2)* test_data["X"] + (1.732/2)* test_data["Y"]
test_data["rot60_Y"] = (1./2)* test_data["Y"] - (1.732/2)* test_data["X"]

test_data["radial_r"] = np.sqrt( np.power(test_data["Y"],2) + np.power(test_data["X"],2) )
test_data.head()

In [ ]:
X = train_data.drop('Category',axis = 1)
y = pd.get_dummies(train_data['Category'])
y.head()

In [ ]:
X = X.astype(float)
X.head()

In [ ]:

X_train,X_test,y_train,y_test=ts(X,y,test_size=0.15,random_state=42)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

In [ ]:
seq.add(Dense(256, input_shape = (X.shape[1],)))
seq.add(Dense(128))
seq.add(Activation('selu'))
seq.add(Dense(128))
seq.add(Activation('selu'))
seq.add(Dropout(0.5))
seq.add(Dense(64))
seq.add(Activation('selu'))
seq.add(Dropout(0.5))
seq.add(Dense(64))
seq.add(Activation('selu'))
seq.add(Dropout(0.5))
seq.add(Dense(64))
seq.add(Activation('selu'))
seq.add(Dense(64))
seq.add(Dropout(0.5))
seq.add(Activation('selu'))
seq.add(Dense(39))
seq.add(Activation('softmax'))
seq.summary()

**Compiling our model in order to fit in the training data**

In [ ]:
seq.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

**Training our model using train_data dataframe**

In [ ]:
train=seq.fit(X_train,y_train, 
         batch_size=32,
         epochs=16,
         verbose=2,
         validation_data=(X_test,y_test))

In [ ]:
test=test_data.drop(['Id'],axis=1)
test=test.astype(float)
test.dtypes

In [ ]:
prediction=seq.predict(test)

In [ ]:
s = np.max(prediction, axis=1).reshape(-1, 1)
predicted = np.array((prediction == s), dtype='int32')
predicted

In [ ]:
col_names=list(sam_sub.columns)
col_names.remove('Id')
print(col_names)

In [ ]:
submission = pd.DataFrame()
submission['Id'] = test_data['Id']
for i , entry in enumerate(col_names):
    submission[entry] = predicted[:,i]

**Getting the prediction dataset**

In [ ]:
submission.head()

In [ ]:
submission.to_csv('../working/submission.csv', index=False)